<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#FUNCTIONS---Run-at-start" data-toc-modified-id="FUNCTIONS---Run-at-start-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>FUNCTIONS - Run at start</a></span><ul class="toc-item"><li><span><a href="#Load-Data" data-toc-modified-id="Load-Data-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Load Data</a></span></li><li><span><a href="#Returns" data-toc-modified-id="Returns-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Returns</a></span></li><li><span><a href="#row_info" data-toc-modified-id="row_info-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>row_info</a></span></li><li><span><a href="#col_info" data-toc-modified-id="col_info-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>col_info</a></span></li><li><span><a href="#Holdings" data-toc-modified-id="Holdings-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Holdings</a></span></li><li><span><a href="#Classify" data-toc-modified-id="Classify-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Classify</a></span></li></ul></li><li><span><a href="#Setup-und-Algo" data-toc-modified-id="Setup-und-Algo-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Setup und Algo</a></span><ul class="toc-item"><li><span><a href="#Multiprocessing" data-toc-modified-id="Multiprocessing-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Multiprocessing</a></span></li></ul></li><li><span><a href="#Analysis-of-resulting-classifications" data-toc-modified-id="Analysis-of-resulting-classifications-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Analysis of resulting classifications</a></span><ul class="toc-item"><li><span><a href="#Error-volatility" data-toc-modified-id="Error-volatility-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Error volatility</a></span></li><li><span><a href="#Distribution-of-classes-per-classification-technique" data-toc-modified-id="Distribution-of-classes-per-classification-technique-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Distribution of classes per classification technique</a></span></li><li><span><a href="#Transition-tables" data-toc-modified-id="Transition-tables-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Transition tables</a></span></li></ul></li><li><span><a href="#Paper" data-toc-modified-id="Paper-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Paper</a></span><ul class="toc-item"><li><span><a href="#Misclassification-table-all-years" data-toc-modified-id="Misclassification-table-all-years-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Misclassification table all years</a></span></li><li><span><a href="#Correct-classification-over-time" data-toc-modified-id="Correct-classification-over-time-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Correct classification over time</a></span></li><li><span><a href="#Size-vs-cap-class" data-toc-modified-id="Size-vs-cap-class-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Size vs cap class</a></span></li><li><span><a href="#Transitions" data-toc-modified-id="Transitions-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Transitions</a></span></li><li><span><a href="#Robustness-test" data-toc-modified-id="Robustness-test-4.5"><span class="toc-item-num">4.5&nbsp;&nbsp;</span>Robustness test</a></span></li><li><span><a href="#Comparing-mean-return-per-class-for-the-different-classification-techniques" data-toc-modified-id="Comparing-mean-return-per-class-for-the-different-classification-techniques-4.6"><span class="toc-item-num">4.6&nbsp;&nbsp;</span>Comparing mean return per class for the different classification techniques</a></span></li></ul></li><li><span><a href="#Sanity-checks" data-toc-modified-id="Sanity-checks-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Sanity checks</a></span><ul class="toc-item"><li><span><a href="#Analysing-individual-portfolios" data-toc-modified-id="Analysing-individual-portfolios-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Analysing individual portfolios</a></span></li><li><span><a href="#Inspecting-individual-nearest-neighbors" data-toc-modified-id="Inspecting-individual-nearest-neighbors-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Inspecting individual nearest neighbors</a></span></li></ul></li></ul></div>

# Classification 

In [ ]:
import feather
import pickle
import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype


from scipy import sparse

import seaborn as sns
import matplotlib.pyplot as plt

from multiprocessing import Pool
from itertools import product

from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import normalize
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import confusion_matrix, classification_report, mean_squared_error

# Progress bar
from ipywidgets import FloatProgress
from IPython.display import display

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

# import logging
# logging.basicConfig(level=logging.DEBUG, format='%(levelname)s - %(message)s')

## FUNCTIONS - Run at start

### Load Data

### Returns
path = '../data/processed/returns.feather'
returns = feather.read_dataframe(path)
returns = returns.rename(columns = {'caldt' : 'report_dt'})
returns = returns.assign(year = returns['report_dt'].dt.year)

### row_info
path = '../data/processed/row_info_f.feather'
row_info = feather.read_dataframe(path)

### col_info
path = '../data/processed/col_info_f.feather'
col_info = feather.read_dataframe(path)

### Holdings
path = '../data/processed/holdings_f.npz'
holdings = sparse.load_npz(path)

In [ ]:
path = '../data/processed/full.pickle'
pickle_off = open(path,"rb")
dict_all_years = pickle.load(pickle_off)
dict_year = dict_all_years[2018]

### Classify

In [ ]:
def classify(row_info_f, holdings_f, param, verbose=False):
    np.random.seed()
    
        
    #### Setup ####
    # Classifier
    neigh = KNeighborsClassifier(n_neighbors = param.loc[0,'n_neighbors'].astype(int), 
                                 p           = 2, 
                                 weights     = 'distance', 
                                 n_jobs      = 1) # distance or uniform
    
    # Data
    X = holdings_f
    y = row_info_f['lipper_class'].values
    n_rows = holdings_f.shape[0]

    # Result dataframe
    style_df = pd.DataFrame({'model_lipper' : y})

    #### Full #### 
    # Predict all at once and save in style_df
#     loo = LeaveOneOut()
#     loo.get_n_splits(X)

#     for train_index, test_index in loo.split(X):
#         X_train, X_test = X[train_index], X[test_index]
#         y_train, y_test = y[train_index], y[test_index]

#         neigh.fit(X_train,y_train)
#         style_df.loc[test_index, 'model_knn_loo'] = neigh.predict(X_test)

    #### Iterative ####
    style_df['model_knn_iterative'] = style_df['model_lipper']
    
    # Index : Setup of index for choosing rows iteratively
    index = np.arange(n_rows)
    np.random.shuffle(index)
    index = np.concatenate((index,index,index,index,index))
    
    n_rows_chosen = round(n_rows * param.loc[0,'perc_rows_used']).astype(int)
    index = index[:n_rows_chosen]
    it = iter(index)
    index = zip(it)
    chosen_indices = []

    # Loop over n_iterations, choose one observation randomly, predict label, save and repeat
    f = FloatProgress(min=0, max=n_rows_chosen)
    if(verbose): 
        display(f)

    for i in index:
        mask = np.arange(X.shape[0]) # mask for whole sample
        mask_is = ~np.isin(mask,i)   # mask to choose all in sample observations
        mask_oos = np.isin(mask,i)   # mask to choose the x out of sample observations for which we predict
        chosen_indices.append(i)

        # Mask X and labels to exclude row for which prediction will be made
        X_sub = X[mask_is]
        y_sub = style_df.loc[mask_is,'model_knn_iterative'].values

        # Fit knn model on all but randomly chosen row
        neigh.fit(X_sub,y_sub) 

        # Predict and save label for randomly chosen row
        style_df.loc[mask_oos,'model_knn_iterative'] = neigh.predict(X[mask_oos])
        f.value += 2

    row_chosen = np.unique(np.array(chosen_indices).flatten()).shape[0]
    #print('Rows randomly chosen:    {:4.2f}%'.format(row_chosen / X.shape[0] * 100))
    #print('Done')
    
    return(style_df)

In [ ]:
def calc_styleadj_returns(row_info_f,returns_f,style_df):

    row_info_m = row_info_f.copy()
    returns_m = returns_f.copy()

    # concat predicted styles to row_info
    row_info_m = pd.concat([row_info_m,style_df],axis = 1)

    # merge predicted styles onto returns
    returns_m = returns_m.merge(row_info_m[[
                                    'crsp_fundno', 'report_dt', 'model_lipper', 'model_knn_loo',
                                    'model_knn_iterative'
                                    ]],
                                    how='left',
                                    on=['crsp_fundno', 'report_dt'])

    # melt the different style columns per model into one (from wide to long)
    returns_m = pd.melt(returns_m,
                                   id_vars=['crsp_fundno', 'report_dt', 'mret'],
                                   value_vars=cols,
                                   var_name='model',
                                   value_name='style')

    # Fill all styles and drop nas

    temp = (returns_m
                                    .groupby(['model','crsp_fundno'])
                                    .apply(lambda x: x.fillna(method = 'ffill'))
    )

    returns_m['style'] = temp['style']
    returns_m = returns_m.dropna()

    # Calc mean return per style
    style_returns = (returns_m
                                    .groupby(['model','style','report_dt'])
                                    .mean()
                                    .reset_index()
                                    .drop(columns='crsp_fundno')
    )

    # Calc cumret per style
    style_returns['cum_ret'] = (style_returns
                                    .assign(cum_ret = lambda x: x.mret + 1)
                                    .groupby(['model','style'])
                                    .apply(lambda x: x['cum_ret'].cumprod())
                                    .reset_index()
                                    ['cum_ret']
    )

    # Merge style returns onto fund returns and calc tracking error
    returns_m = (returns_m
                                    .rename(columns = {'mret' : 'fund_ret'}) 
                                    .merge(style_returns,
                                                how = 'left',
                                                on = ['model','style','report_dt'])
                                    .assign(error = lambda df: df['fund_ret'] - df['mret'])
                                    .rename(columns = {'mret' : 'style_ret',
                                                       'cum_ret' : 'style_cum'}) 
    )

    returns_m = returns_m[['crsp_fundno', 'report_dt', 'model', 'style',
                           'fund_ret', 'style_ret', 'style_cum', 'error']]
    
    return(returns_m, style_returns)

In [ ]:
def error_vola_deciles(returns_m): 
    error_vol = (returns_m
                 .groupby(['crsp_fundno','model'])['error']
                 .std()
                 .reset_index())
    error_vol['error'] = error_vol['error'] * 100
    
    error_vol = (error_vol
                .groupby('model')[['error']]
                .apply(lambda x : x.quantile(np.round(np.arange(0.1,1,0.1),2)))
                .reset_index()
                .pivot(columns='level_1',values='error',index='model'))
    return(error_vol)

In [ ]:
def weighted_average_score_new(param_grid, relevant_params, measures):

    param_grid.reset_index()
    param_grid = param_grid.fillna(value=0)
    param_grid['param_id'] = (param_grid
                                  .groupby(relevant_params)
                                  .ngroup())
    
    # Fix for issue with same param_id for lipper rows
    lipper_rows = param_grid.loc[param_grid['model'] == 'lipper',:].copy()
    lipper_rows['param_id'] = lipper_rows.groupby(['preprocessing']).ngroup()
    lipper_rows['param_id'] = (lipper_rows['param_id'] + 1) * -1
    param_grid.loc[param_grid['model'] == 'lipper'] = lipper_rows

    scores = param_grid[measures]
    params_only = param_grid.drop(columns = measures)
    
    weights = (param_grid[['year','count']]
                                  .drop_duplicates()
                                  .assign(weight = lambda x: x['count'] / np.sum(x['count'])))
    weights = weights[['weight']].values

    scores = scores.groupby(params_only['param_id']).apply(lambda x: np.sum(x * weights))

    params_only = (params_only
                      .drop_duplicates(relevant_params)
                      .drop(columns = ['year','count']))

    result = params_only.merge(scores, how = 'left', on = 'param_id')
    
    return(result)

In [ ]:
def weighted_average_score(results): 
    deciles = results.iloc[:,0:9]
    deciles = deciles.reset_index(drop = True)

    param_grid = results.iloc[:,9:]
    param_grid = param_grid.reset_index()

    weights = (param_grid[['year','count']]
                                  .drop_duplicates()
                                  .assign(weight = lambda x: x['count'] / np.sum(x['count'])))
    weights = weights[['weight']].values

    param_grid['param_id'] = (param_grid
                                  .groupby(['model','perc_rows_used', 'distance_param', 'n_neighbors', 'weights'])
                                  .ngroup())
    
    deciles = deciles.groupby(param_grid['param_id']).apply(lambda x: np.sum(x * weights))


    param_grid = (param_grid
                      .drop_duplicates(['model','perc_rows_used', 'distance_param', 'n_neighbors', 'weights'])
                      .drop(columns = ['year','count']))

    result = param_grid.merge(deciles, how = 'left', on = 'param_id')
    
    return(result)

In [ ]:
def full_algo(year,perc_rows_used,distance_param):
        
    param = pd.DataFrame(np.array([[year, perc_rows_used, n_neighbors]]),
                   columns = ['year','perc_rows_used','n_neighbors'])
    
    
    np.random.seed(0)
    
    row_info_f = dict_all_years[param.loc[0,'year']]['row_info_f']
    returns_f = dict_all_years[param.loc[0,'year']]['returns_f']
    holdings_f = dict_all_years[param.loc[0,'year']]['holdings_ft']
    begin_date = dict_all_years[param.loc[0,'year']]['begin_date']
    end_date = dict_all_years[param.loc[0,'year']]['end_date']
    logging.debug('Data loaded and filtered')
    
    style_df = classify(row_info_f, holdings_f, param, verbose=False)
    logging.debug('Classification complete')
    
    returns_m, style_returns = calc_styleadj_returns(row_info_f, returns_f, style_df)
    logging.debug('Style calculated')
    
    temp = error_vola_deciles(returns_m)
    logging.debug('Deciles calculated')
    logging.info('Algo completed')
    
    temp['year']           = param.loc[0,'year']
    temp['count']          = row_info_f.shape[0]
    temp['perc_rows_used'] = param.loc[0,'perc_rows_used']
    temp['n_neighbors']    = param.loc[0,'n_neighbors']
    
    return(temp)

In [ ]:
def expand_grid(dictionary):
    temp = pd.DataFrame([row for row in product(*dictionary.values())], 
                           columns=dictionary.keys())
    return(temp)

## Setup und Algo

In [ ]:
#### Options #####
##################
style_class      = 'lipper_class' # Choose lipper_class, style_class or cap_class
cols             = ['model_lipper',
                    'model_knn_loo',
                    'model_knn_iterative'] # Do not change, only names

In [ ]:
full = [2010,2011,2012,2013,2014,2015]
param_grid = dict(year           = [2015], # Integer or string 'full' for the whole sample
                  perc_rows_used = [1],         # 1: all funds are reclassified once, 2: ... twice, etc.
                  n_neighbors    = [10]        # Number of neighbors to use in k-nn algorithm
                 )

param_grid = expand_grid(param_grid)
param_tuples = list(param_grid.itertuples(index=False,name=None))

print('Number of iterations:       {}'.format(param_grid.shape[0]))

### Multiprocessing

In [ ]:
with Pool() as pool:
    results = pool.starmap(full_algo, param_tuples)
results = pd.concat(results)
print('Finished')

In [ ]:
results

In [ ]:
data = weighted_average_score(results).sort_values(0.5).drop(columns='param_id')
data_graph = data.query('''distance_param == 2 and weights == 'distance' ''')

In [ ]:
data.drop_duplicates(['model',0.5])

In [ ]:
sns.lineplot(data = data, y = 0.5, x = 'n_neighbors', hue = 'model')

## Analysis of resulting classifications

### Error volatility

In [ ]:
full = [2010,2011,2012,2013,2014,2015,2016]
param_grid = dict(year           = [2016],     # Integer or string 'full' for the whole sample
                  perc_rows_used = [1],          # 1: all funds are reclassified once, 2: ... twice, etc.
                  n_neighbors    = [15],         # Number of neighbors to use in k-nn algorithm
                 )

param = expand_grid(param_grid)

In [ ]:
row_info_f = dict_all_years[param.loc[0,'year']]['row_info_f']
returns_f = dict_all_years[param.loc[0,'year']]['returns_f']
holdings_f = dict_all_years[param.loc[0,'year']]['holdings_ft']
begin_date = dict_all_years[param.loc[0,'year']]['begin_date']
end_date = dict_all_years[param.loc[0,'year']]['end_date']

In [ ]:
end_date

In [ ]:
style_df = classify(row_info_f, holdings_f, param, verbose=True)

In [ ]:
returns_m, style_returns = calc_styleadj_returns(row_info_f, returns_f, style_df)

temp = error_vola_deciles(returns_m)

In [ ]:
error_vol = (returns_m
                 .groupby(['crsp_fundno','model'])['error']
                 .std()
                 .reset_index())

error_vol['error'] = error_vol['error'] * 100
error_vol.groupby('model')['error'].describe()

In [ ]:
fig, ax = plt.subplots(figsize=(20,5))
for i, col in enumerate(cols):
    ax = error_vol.query(''' model == @col and error < 3''')['error'].hist(label = col,bins = 100, alpha = 0.5)

ax.set_xlim(0,3)    
fig.legend()
plt.show()

### Distribution of classes per classification technique

In [ ]:
print('Overlap between Lipper class and: \n')
print('Knn full prediction:         {:2.2f}%'
      .format(np.sum(style_df['model_lipper'] == style_df['model_knn_loo']) / len(style_df.index) * 100))
print('Knn iterative prediction:    {:2.2f}%'
      .format(np.sum(style_df['model_lipper'] == style_df['model_knn_iterative']) / len(style_df.index) * 100))

data = style_df.apply(pd.Series.value_counts, normalize = True)
data = data.assign(style = data.index)
data = data.melt(id_vars = 'style', value_vars = data.columns[:-1])

sns.set()
plt.figure(figsize=(18,10))
g = sns.barplot(data = data, y = 'style', x = 'value', hue = 'variable')

plt.title('Style distribution')
plt.ylabel('Style')
plt.xlabel('Percentage')

plt.show()

### Transition tables

In [ ]:
temp = round(pd.crosstab(style_df['model_lipper'], 
                         style_df['model_knn_iterative'], 
                         margins=True, normalize='columns') * 100,2)
temp

## Paper

### Misclassification table all years

In [ ]:
years = [2010,2011,2012,2013,2014,2015,2016,2017,2018]

results = []
weights = []
row_info_new_full = [] 

for year in years:
    param_grid = dict(year           = [year],     # Integer or string 'full' for the whole sample
                      perc_rows_used = [2],          # 1: all funds are reclassified once, 2: ... twice, etc.
                      n_neighbors    = [15],         # Number of neighbors to use in k-nn algorithm
                 )

    param = expand_grid(param_grid)

    row_info_f = dict_all_years[param.loc[0,'year']]['row_info_f']
    returns_f = dict_all_years[param.loc[0,'year']]['returns_f']
    holdings_f = dict_all_years[param.loc[0,'year']]['holdings_ft']
    begin_date = dict_all_years[param.loc[0,'year']]['begin_date']
    end_date = dict_all_years[param.loc[0,'year']]['end_date']

    style_df = classify(row_info_f, holdings_f, param, verbose=True)
    
    row_info_f['crsp_fundno'].reset_index(inplace = True, drop = True)
    style_df.reset_index(inplace = True, drop = True)

    row_info_new = pd.concat([row_info_f['crsp_fundno'], style_df], axis = 1)
    row_info_new['report_dt'] = begin_date

    row_info_new_full.append(row_info_new)

    results.append(pd.crosstab(style_df['model_lipper'], 
                         style_df['model_knn_iterative'],
                         normalize='columns') * 100)
    
    weights.append(pd.crosstab(style_df['model_lipper'], 
                         style_df['model_knn_iterative']))
    

row_info_new_full = pd.concat(row_info_new_full)

In [ ]:
full_weight = (weights[0] + weights[1] + weights[2] + weights[3] + weights[4] + weights[5]+ weights[6]
        + weights[7] + weights[8])
full_weight = full_weight.T

full_perc = full_weight / full_weight.sum()
full_perc = full_perc.T

In [ ]:
full_perc

In [ ]:
print(round(full_perc,2).to_latex(index = True,
                    index_names = False,
                    bold_rows = False)
     )

In [ ]:
# Percentage correctly classified
sum(np.diagonal(full_weight)) / np.sum(full_weight.values)

### Correct classification over time

In [ ]:
temp = []
temp_full = []

for i in np.arange(0,9):
    temp.append(np.diagonal(weights[i]) / weights[i].sum(1))
    temp_full.append(sum(np.diagonal(weights[i])) / np.sum(weights[i].values))
    
class_years = pd.concat(temp, axis = 1)
class_years.index = class_years.index.astype(str)

temp_full = pd.DataFrame(temp_full).T
temp_full.columns = class_years.columns
temp_full.index = ['Total']

class_years = pd.concat([class_years,temp_full])
class_years.columns = years
class_years

In [ ]:
print(round(class_years,2).to_latex(index = True,
                    index_names = False,
                    bold_rows = False)
     )

### Size vs cap class

In [ ]:
cap_style = full_weight.copy()

names = ['EI', 'LC', 'LC', 'LC', 'ML', 'ML', 'ML',
                  'MC', 'MC', 'MC', 'SC', 'SC', 'SC']

cap_style.index = names
cap_style.columns = names
cap_style = cap_style.groupby(level=0).sum()
cap_style = cap_style.groupby(level=0, axis = 1).sum()

cap_perc = cap_style / cap_style.sum()
cap_perc = cap_perc.T
cap_perc

In [ ]:
print(round(cap_perc,2).to_latex(index = True,
                    index_names = False,
                    bold_rows = False)
     )

In [ ]:
cap_style = full_weight.copy()

names = ['EI', 'V', 'C', 'G', 'V', 'C', 'G',
                  'V', 'C', 'G', 'V', 'C', 'G']

cap_style.index = names
cap_style.columns = names
cap_style = cap_style.groupby(level=0).sum()
cap_style = cap_style.groupby(level=0, axis = 1).sum()

style_perc = cap_style / cap_style.sum()
style_perc = style_perc.T
style_perc

In [ ]:
print(round(style_perc,2).to_latex(index = True,
                    index_names = False,
                    bold_rows = False)
     )

### Transitions

In [ ]:
row_info = row_info_new_full.copy()

In [ ]:
row_info.columns = ['crsp_fundno','lipper_class','new_lipper_class','report_dt']

row_info['cap_class'] = row_info['lipper_class'].astype(str).str[0:2]
row_info['style_class'] = row_info['lipper_class'].astype(str).str[2]

row_info['new_cap_class'] = row_info['new_lipper_class'].astype(str).str[0:2]
row_info['new_style_class'] = row_info['new_lipper_class'].astype(str).str[2]

cat_type_style = CategoricalDtype(categories = ['E', 'V', 'C', 'G'], ordered=True)
cat_type_cap = CategoricalDtype(categories = ['SC', 'MC', 'ML', 'EI', 'LC'], ordered=True)

row_info['style_class'] = row_info['style_class'].astype(cat_type_style)
row_info['cap_class'] = row_info['cap_class'].astype(cat_type_cap)

row_info['new_style_class'] = row_info['new_style_class'].astype(cat_type_style)
row_info['new_cap_class'] = row_info['new_cap_class'].astype(cat_type_cap)

In [ ]:
obj_per_portno = row_info[['crsp_fundno','new_lipper_class']]
ax = (obj_per_portno['new_lipper_class']
    .groupby([
        obj_per_portno['crsp_fundno']
    ])
    .nunique()
    .value_counts()
    .sort_values()
    .plot(kind='barh',
          color = 'b',
          figsize=(18,5),
          title='Number of unique objective codes per crsp_portno')
)

In [ ]:
lagged_class = row_info[['crsp_fundno','report_dt','new_lipper_class']]
lagged_class = lagged_class.sort_values(by=['crsp_fundno','report_dt'])
lagged_class = lagged_class.reset_index(drop = True)

lagged_class = lagged_class.assign(lag_lipper_class = lagged_class.new_lipper_class.shift())
lagged_class = lagged_class.dropna()

mask = lagged_class.groupby(by = 'crsp_fundno').head(1).index

lagged_class = lagged_class.loc[~lagged_class.index.isin(mask)]

switch = round(pd.crosstab(lagged_class.new_lipper_class, lagged_class.lag_lipper_class,margins = True))
switch

In [ ]:
print(switch.to_latex(index = True,
                    index_names = False,
                    bold_rows = False)
     )

In [ ]:
switch= switch.iloc[0:13,0:13]
print(switch.sum().sum() - np.diag(switch).sum())

In [ ]:
lagged_class = row_info[['crsp_fundno','report_dt','lipper_class']]
lagged_class = lagged_class.sort_values(by=['crsp_fundno','report_dt'])
lagged_class = lagged_class.reset_index(drop = True)

lagged_class = lagged_class.assign(lag_lipper_class = lagged_class.lipper_class.shift())
lagged_class = lagged_class.dropna()

mask = lagged_class.groupby(by = 'crsp_fundno').head(1).index

lagged_class = lagged_class.loc[~lagged_class.index.isin(mask)]

switch = round(pd.crosstab(lagged_class.lipper_class, lagged_class.lag_lipper_class, margins = True) )
switch

In [ ]:
print(switch.to_latex(index = True,
                    index_names = False,
                    bold_rows = False)
     )

In [ ]:
switch= switch.iloc[0:13,0:13]
print(switch.sum().sum() - np.diag(switch).sum())

In [ ]:
switch.sum().sum()

In [ ]:
lagged_class = row_info[['crsp_fundno','report_dt','new_style_class']]
lagged_class = lagged_class.sort_values(by=['crsp_fundno','report_dt'])
lagged_class = lagged_class.reset_index(drop = True)

lagged_class = lagged_class.assign(lag_new_style_class = lagged_class.new_style_class.shift())
lagged_class = lagged_class.dropna()

mask = lagged_class.groupby(by = 'crsp_fundno').head(1).index

lagged_class = lagged_class.loc[~lagged_class.index.isin(mask)]

lag_style = pd.crosstab(lagged_class.new_style_class, lagged_class.lag_new_style_class)
lag_style

In [ ]:
print(lag_style.to_latex(index = True,
                    index_names = False,
                    bold_rows = True)
     )

In [ ]:
lagged_class = row_info[['crsp_fundno','report_dt','new_cap_class']]
lagged_class = lagged_class.sort_values(by=['crsp_fundno','report_dt'])
lagged_class = lagged_class.reset_index(drop = True)

lagged_class = lagged_class.assign(lag_new_cap_class = lagged_class.new_cap_class.shift())
lagged_class = lagged_class.dropna()

mask = lagged_class.groupby(by = 'crsp_fundno').head(1).index

lagged_class = lagged_class.loc[~lagged_class.index.isin(mask)]

lag_cap = pd.crosstab(lagged_class.new_cap_class, lagged_class.lag_new_cap_class)
lag_cap

In [ ]:
lagged_class.query('''new_cap_class == 'LC' and lag_new_cap_class == 'SC' ''')

In [ ]:
row_info.query('''crsp_fundno == 6642 ''')

In [ ]:
print(lag_cap.to_latex(index = True,
                    index_names = False,
                    bold_rows = False)
     )

### Robustness test

In [ ]:
k_values  = [5,15,30,50]
row_values  = [1,2,3]

for k in k_values:
    years = [2010,2011,2012,2013,2014,2015,2016,2017,2018]
    years = [2010]

    results = []
    weights = []

    for year in years:
        param_grid = dict(year           = [year],     # Integer or string 'full' for the whole sample
                          perc_rows_used = [k],          # 1: all funds are reclassified once, 2: ... twice, etc.
                          n_neighbors    = [5],         # Number of neighbors to use in k-nn algorithm
                     )

        param = expand_grid(param_grid)

        row_info_f = dict_all_years[param.loc[0,'year']]['row_info_f']
        returns_f = dict_all_years[param.loc[0,'year']]['returns_f']
        holdings_f = dict_all_years[param.loc[0,'year']]['holdings_ft']
        begin_date = dict_all_years[param.loc[0,'year']]['begin_date']
        end_date = dict_all_years[param.loc[0,'year']]['end_date']

        style_df = classify(row_info_f, holdings_f, param, verbose=False)

        weights.append(pd.crosstab(style_df['model_lipper'], 
                             style_df['model_knn_iterative']))
        
    temp = (weights[0] + weights[1])
#    temp = (weights[0] + weights[1] + weights[2] + weights[3] + weights[4] + weights[5]+ weights[6]
#        + weights[7] + weights[8])
    temp_perc = temp / temp.sum()
        
    print('k: {}'.format(k))
    print('Avg. correctly classified: {}'.format(sum(np.diagonal(temp)) / np.sum(temp.values)))


### Comparing mean return per class for the different classification techniques

In [ ]:
fig, ax = plt.subplots(figsize=(20,5),ncols=3, sharey='row')
for i, col in enumerate(cols):
    sns.lineplot(data = style_returns.query(''' model == @col '''),
                 x='report_dt', y='cum_ret', hue='style', ax=ax[i])

# Subplot titles
title = cols
ax[0].set_ylabel('Cumulative return per class')

for i in range(0,3):
    ax[i].set_title(title[i], fontsize = 16)
    ax[i].set_xlabel('')
    for label in ax[i].get_xticklabels():
        label.set_rotation(45)
        
for i in range(1,3):
    ax[i].get_legend().remove()

plt.show()

## Sanity checks

### Analysing individual portfolios

In [ ]:
comparing_df.query('''true == 'V' and iterative_5 == 'G' ''').head()

In [ ]:
crsp_fundno = 18307
most_common_stocks_fund(year=2017, crsp_fundno=crsp_fundno)

In [ ]:
def most_common_stocks_fund(crsp_fundno,row_info,year):
    "This prints a passed string into this function"
    # Enter date for which most common holdings are calculated
    year = year
    crsp_fundno = crsp_fundno
    row_info_l = row_info

    holdings_coo = holdings.tocoo()

    df_sparse = pd.DataFrame({'row'  : holdings_coo.row,
                              'col'  : holdings_coo.col,
                              'data' : holdings_coo.data})

    df_sparse = df_sparse.merge(row_info_l[['year','row','crsp_fundno']],how='left',on='row')
    my_filter = '''year == @year and crsp_fundno == @crsp_fundno '''
    no_unique_funds = row_info_l.query(my_filter).shape[0]

    sum_col = (df_sparse
               .query(my_filter)
               .groupby(by = ['col'])
               .mean()
               .sort_values('data',ascending = False)
               .join(col_info[['security_name','col']],how='left')
               .assign(percent = lambda x:  x.data)
               .drop(columns=['row','data','col','year','crsp_fundno'])
               .reset_index(drop=True)
               .head(10))
    
    print(
        'Average of most held stocks for one fund in one year: ','\n\n'
        '{}'.format(row_info.query('crsp_fundno == @crsp_fundno').iloc[0,2]),'\n\n'
        'crsp_fundno:                            {}'.format(crsp_fundno),'\n'
        'Number of observations in that year:    {}'.format(no_unique_funds))

    return sum_col

### Inspecting individual nearest neighbors

In [ ]:
neigh.kneighbors(X[1234],n_neighbors = n_neighbors)

In [ ]:
def name_nearestneighbors(row_info,neigh,i,n_neighbors = 5):
    print('Name:')
    print(row_info.loc[i].fund_name)
    print(row_info.loc[i].crsp_fundno)
    print('\nNearest Neighbors:')
    nn_index = neigh.kneighbors(X[i],n_neighbors = n_neighbors)[1].flatten()
    nn_names = row_info.loc[nn_index].fund_name.values
    nn_fundno = row_info.loc[nn_index].crsp_fundno.values
    
    for name in nn_names[1:]:
        print(name)

In [ ]:
name_nearestneighbors(row_info,neigh,i = 1234, n_neighbors = 5)

In [ ]:
crsp_fundno = 36608
most_common_stocks_fund(crsp_fundno=crsp_fundno)

In [ ]:
crsp_fundno = 3690
most_common_stocks_fund(crsp_fundno=crsp_fundno)